## Import Packages

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import time
import numpy as np
import pandas as pd 

## Get List of Major URLs

In [8]:
# Open chrome brower and go to parent website containing all majors
driver = webdriver.Chrome()
driver.get('https://www.payscale.com/college-roi/major')

#Isolate body containing all college major info
majors = driver.find_elements_by_xpath('//*[@class="col-xs-12 col-sm-6"]/p')

#Generate nested list containing all college major names and specific urls
major_urls = []
for major in majors:
    #Isolate major name and url
    major_name = major.find_element_by_xpath( './/a' ).text.lower().replace('majors', '').strip()
    major_url = major.find_element_by_xpath( './/a' ).get_attribute("href")
    
    #Store items
    major_urls.append([major_name, major_url])

## Prep Repository DF For State ROI Input

In [9]:
repo_df = pd.DataFrame({'major': [],\
                        'college_name':[],\
                        'college_type':[],\
                        'financial_aid':[],\
                        'roi':[],\
                        'total_cost':[]})

## Scrape Each State

In [10]:
counter = 0
for url in major_urls: 
  
    #Unpack major-specific list containing name and url
    major_name, major_url = url
    print(f"{major_name}: initiate")
    
    #Go to major-specific url
    driver.get( major_url ) #enter state-specific url
    
    # Click review button to view complete dataset
    oncampus_button = driver.find_element_by_xpath('//*[@id="roi-list-root"]/div/div/div/div/button')
    oncampus_button.click()

    roi_type_button = driver.find_element_by_xpath('//div[@id="roi-list-root"]/div/div/div[3]//button[1]')
    roi_type_button.click()

    load_more_button = driver.find_element_by_xpath('//*[@class="col-md-4 offset-md-2"]')
    load_more_button.click()
    
    financial_aid_button = driver.find_element_by_xpath('//*[@id="roi-list-root"]/div/div/div[2]//button[2]')
    financial_aid_button.click() 
    
    #Wait for entire ROI table to load
    wait_colleges = WebDriverWait( driver, 10 )
    colleges = wait_colleges.until( EC.presence_of_all_elements_located((By.XPATH, \
                                '//*[@class="table college-roi-table table-bordered table-striped table-condensed"]/tbody/tr')) )

    #Scrape info per row (ie. college)
    for college in colleges:

        #Define additional xpaths and scrape college information
        text = college.find_element_by_xpath('./td[2]//div').text #roughly extract text
        college_name =  college.find_element_by_xpath('./td[2]//div').text #join college name
        college_type = re.sub( r'[)$,%(-]','', college_name.lower().split(r'(')[-1]) #clean up college type
        total_cost = re.sub( r'[)$,%(-]','', college.find_element_by_xpath('./td[11]/span').text ) 
        roi = re.sub( r'[$,%]','', college.find_element_by_xpath('./td[9]/span').text )


        #Store all college data into dataframe
        college_df = pd.DataFrame({'major': [major_name],\
                        'college_name':[college_name],\
                        'college_type':[college_type],\
                        'financial_aid': [True],\
                        'roi':[roi],\
                        'total_cost':[total_cost]})

        #Add college_df to repository dataframe
        repo_df = pd.concat([repo_df, college_df],ignore_index=True, axis=0)

        #Keep track of number of colleges
        counter += 1
            
    print(f"{major_name}: complete")
    print('='*25,counter, '='*25)

driver.close()

art: initiate
art: complete
========================= 686 =========================
business: initiate
business: complete
========================= 2261 =========================
computer science and math: initiate
computer science and math: complete
========================= 3170 =========================
economics: initiate
economics: complete
========================= 3458 =========================
education: initiate
education: complete
========================= 3812 =========================
engineering: initiate
engineering: complete
========================= 4348 =========================
humanities: initiate
humanities: complete
========================= 5099 =========================
nursing: initiate
nursing: complete
========================= 5565 =========================
philosophy and religious studies: initiate
philosophy and religious studies: complete
========================= 5595 =========================
political science: initiate
political science: complete
======

## Export Repository DF As CSV

In [11]:
repo_df.to_csv('college_roi_bymajor.csv', index=None, header=True)